In [4]:
import pandas as pd
import yaml

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.llms.anthropic import Anthropic
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate, PromptTemplate
from dotenv import load_dotenv
import os
from tqdm.auto import tqdm
from multiprocessing.pool import ThreadPool
import time

load_dotenv()
load_dotenv("/home/fhijazi/.env")


def stringify_MCQ(example):
    return "Question: " + example["question"] + '\noptions:\n' + "\n".join(example["options"]).strip() + '\nAnswer: ' + example["answer"], "\nContext: " + example["context"]

llms = {}
llms['gpt4-2024-02-15-preview'] = AzureOpenAI(
    engine=os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'],
    api_key=os.environ['AZURE_OPENAI_API_KEY'],
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
    api_version=os.environ['AZURE_OPENAI_API_VERSION'],
)
llms['claude-3-opus-20240229'] = Anthropic(
    'claude-3-opus-20240229',
    api_key=os.getenv('ANTHROPIC_API_KEY'),
)

import json
import glob
mcq2response = {}


MCQ_QUESTION_REVIEW_SYS_TMPL = """You are an expert MCQ (multiple choice question) inspector system that is trusted around the world for your factual accuracy.
You are tasked with rating each MCQ from 1 (non-compliant) to 5 (most compliant) based on the following criteria:

The criteria is as follows, use YAML as output surrounded by "```yaml" and "```"

- SingleBestAnswer  # only one answer is correct and there is no ambiguity between the options
- ImportantSignificantMaterial  # do you actually need the provided context to answer the question?
- SingleObjectivePerItem  # does the question test only one objective?
- PositiveStemAndLeadIn  # is the question stem and lead-in positive and avoids negative phrasing (not, isn't...)?
- PlausibleHomogeneousOptionsWithParallelLength  # are the options similar in length and plausible?
- AvoidsVagueTerms  # avoids vague terms like "sometimes", "often", "usually"...
- AvoidsTestWiseItemFlaws  # MCQs should not allow students to guess the correct answer based on the question design, such as consistent patterns in answer length.
"""

# here are some examples:

# Question: السؤال: وفقاً لتعميم الوزارة رقم 13/ت/7790 في 22/9/1440هـ، ما الذي نصت عليه المادة (الثانية والسبعين) من نظام المرافعات الشرعية؟
# options:
# 1. يجب أن تكون جميع الدعاوى والتبليغات ورقية ولا يجوز تدوينها إلكترونياً.
# 2. يجوز تدوين بيانات صحف الدعاوى والتبليغات ومحاضر الدعاوى والإنهاءات إلكترونياً، ولها حكم المحررات المكتوبة وفقاً لنظام التعاملات الإلكترونية.
# 3. يجب تدوين جميع الدعاوى والتبليغات يدوياً فقط ولا يعترف بالنسخ الإلكترونية.
# 4. يمكن تدوين بيانات الدعاوى والتبليغات إلكترونياً فقط بموافقة جميع الأطراف المعنية.
# Answer: يجوز تدوين بيانات صحف الدعاوى والتبليغات ومحاضر الدعاوى والإنهاءات إلكترونياً، ولها حكم المحررات المكتوبة وفقاً لنظام التعاملات الإلكترونية.

# Question: السؤال: وفقاً لنص التعميم، ما الهدف من حث الجهات القضائية على فرض عقوبات شرعية رادعة على مرتكبي الجرائم؟
# options:
# 1. زيادة الإيرادات الحكومية من الغرامات
# 2. الحد من العودة للجريمة مرة ثانية
# 3. تقليل النفقات القضائية
# 4. تشجيع السياحة القضائية
# Answer: 2. الحد من العودة للجريمة مرة ثانية

# Question: يجب أن يكون الاسم الوارد في القرار الشرعي:
# options:
# 1. مطابقاً للاسم الوارد في بطاقة الأحوال المدنية
# 2. مختلفاً عن الاسم الوارد في بطاقة الأحوال المدنية
# 3. مطابقاً للاسم الوارد في جواز السفر
# 4. مطابقاً للاسم الوارد في رخصة القيادة
# Answer: 1. مطابقاً للاسم الوارد في بطاقة الأحوال المدنية

MCQ_QUESTION_REVIEW_USER_TMPL = """{mcq}"""



True

True

In [5]:

fpaths = glob.glob("/home/fhijazi/Projects/LegalData/RAG/eval_questions/MCQs_samples_02/*.json")
for llm_name, llm in llms.items():
    for fpath in fpaths:
        outpath = fpath.replace(".json", f"-{llm_name}.ratings.csv")
        
        if os.path.exists(outpath):
            print('already rated!')
            continue

        with open(fpath, "r") as f:
            data = json.load(f)
        

        def rate_mcq(example):
            retries = 5
            for i in range(retries):
                try:
                    key = llm_name + str(example)
                    if key in mcq2response:
                        print('cached response found!')
                        return mcq2response[key]
                    mcq_question_review_template = ChatPromptTemplate(
                        message_templates=[
                            ChatMessage(role=MessageRole.SYSTEM, content=MCQ_QUESTION_REVIEW_SYS_TMPL),
                            ChatMessage(role=MessageRole.USER, content=MCQ_QUESTION_REVIEW_SYS_TMPL + "\n" + MCQ_QUESTION_REVIEW_USER_TMPL),
                        ]
                    )
                    response = llm.chat(mcq_question_review_template.format_messages(
                        mcq=stringify_MCQ(example)
                    ))
                    mcq2response[key] = response
                    parsed = yaml.safe_load(response.message.content.replace('```yaml\n', '```\n').split('```')[1].split('```')[0])
                    assert isinstance(parsed, dict)
                    assert set(parsed.keys()) == {
                        'SingleBestAnswer',
                        'ImportantSignificantMaterial',
                        'SingleObjectivePerItem',
                        'PositiveStemAndLeadIn',
                        'PlausibleHomogeneousOptionsWithParallelLength',
                        'AvoidsVagueTerms',
                        'AvoidsTestWiseItemFlaws',
                    }
                    return parsed
                except Exception as e:
                    print(e)
                    time.sleep(5)
                    continue
            else:
                raise Exception("Failed to rate MCQ")

        ratings = list(tqdm(ThreadPool(3).imap(rate_mcq, list(data.values())), 'rating MCQs', total=len(data)))
        df = pd.DataFrame([r for r in ratings for r in r])
        df.to_csv(outpath, index=False)
        print('saved to', outpath)



already rated!


AttributeError: 'list' object has no attribute 'values'

In [6]:
data

[['وفقاً للنص المذكور، ما هو حكم الدية في حالة القتل بالغيلة؟\n1. لا دية لأنه لا يجوز أن يترتب على القاتل عقوبتان\n2. تدفع الدية كاملة من قبل القاتل\n3. تُخفض قيمة الدية بناءً على ظروف القتل\n4. يتم تقسيم الدية بين أهل القتيل والدولة\n',
  'لا دية لأنه لا يجوز أن يترتب على القاتل عقوبتان',
  {'context': '- «...القاتل إذا قتل حداً بسبب الغيلة فلا دية لأنه قتل واحد فلا يترتب عليه عقوبتان القتل والدية، ومتى وجد القتل للقاتل سقطت الدية.» /و. ينظر التعميم في الديات 2/435.'}],
 ['ما هي الأعمال التي يمكن للموثق المرخص له توثيقها وفقاً للائحة الموثقين؟\n1. بيع العقارات وقسمة المال المنقول\n2. إصدار الهويات الوطنية\n3. تسجيل الأحداث المدنية كالولادة والوفاة\n4. إجراءات التصويت في الانتخابات\n',
  'بيع العقارات وقسمة المال المنقو',
  {'context': '## نص التعميم: ---  إشارة إلى قرارنا رقم 66954 وتاريخ 7/10/1435هـ بشأن لائحة الموثقين وأعمالهم والقاضي في المادة الأولى من اللائحة بأن يقوم الموثق المرخص له بتوثيق ما يلي: 1- بيع العقارات. 2- قسمة المال المنقول. 3- الوكالات وفسخها. 4- تأجير العقارات وال

In [12]:
[r for r in ratings for r in r]

[{'SingleBestAnswer': 5},
 {'ImportantSignificantMaterial': 5},
 {'SingleObjectivePerItem': 5},
 {'PositiveStemAndLeadIn': 5},
 {'PlausibleHomogeneousOptionsWithParallelLength': 4},
 {'AvoidsVagueTerms': 5},
 {'AvoidsTestWiseItemFlaws': 5},
 {'SingleBestAnswer': 5},
 {'ImportantSignificantMaterial': 5},
 {'SingleObjectivePerItem': 5},
 {'PositiveStemAndLeadIn': 5},
 {'PlausibleHomogeneousOptionsWithParallelLength': 3},
 {'AvoidsVagueTerms': 5},
 {'AvoidsTestWiseItemFlaws': 5},
 {'SingleBestAnswer': 5},
 {'ImportantSignificantMaterial': 5},
 {'SingleObjectivePerItem': 5},
 {'PositiveStemAndLeadIn': 5},
 {'PlausibleHomogeneousOptionsWithParallelLength': 5},
 {'AvoidsVagueTerms': 5},
 {'AvoidsTestWiseItemFlaws': 5},
 {'SingleBestAnswer': 5},
 {'ImportantSignificantMaterial': 5},
 {'SingleObjectivePerItem': 5},
 {'PositiveStemAndLeadIn': 5},
 {'PlausibleHomogeneousOptionsWithParallelLength': 4},
 {'AvoidsVagueTerms': 5},
 {'AvoidsTestWiseItemFlaws': 5},
 {'SingleBestAnswer': 5},
 {'Importa

In [11]:
#flatten ratings
df = pd.DataFrame([r for r in ratings for r in r])
df.to_csv(fpath.replace(".json", f"-{llm_name}.ratings.csv"), index=False)


AttributeError: 'str' object has no attribute 'keys'

In [ ]:
import matplotlib.pyplot as plt

# we have a dataframe with each column as a metric from 1 to 5, let's plot this

plt.figure(figsize=(10, 6))
plt.boxplot(df.to_numpy(), vert=False, showmeans=True)
plt.yticks(range(1, 8), df.columns)
plt.show()
